In [185]:
import pandas as pd
import numpy as np


In [186]:
df=pd.read_csv("Churn_Dataset.csv")

In [187]:
from sklearn.preprocessing import LabelEncoder 
le=LabelEncoder()


In [188]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [189]:

!pip install xgboost shap imbalanced-learn --quiet


In [190]:
df["TotalCharges"]

0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: TotalCharges, Length: 7043, dtype: object

In [191]:
df["TotalCharges"]=pd.to_numeric(df["TotalCharges"],errors="coerce")

In [192]:
df=df.drop("customerID",axis=1)


In [193]:
df['Churn']=le.fit_transform(df['Churn'])

In [194]:
df.isna().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [195]:
if df["TotalCharges"].isna().sum() > 0:
    median_val = df["TotalCharges"].median()
    df["TotalCharges"].fillna(median_val, inplace=True)

C:\Users\tarun\AppData\Local\Temp\ipykernel_10500\2737330427.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["TotalCharges"].fillna(median_val, inplace=True)


In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


In [197]:
num_cols=["tenure","MonthlyCharges","TotalCharges"]
cat_cols=[c for c in df.columns if c not in num_cols + ['Churn']]
print("Numeric:",num_cols)
print("Categorical:",cat_cols)


Numeric: ['tenure', 'MonthlyCharges', 'TotalCharges']
Categorical: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


In [198]:
from sklearn.model_selection import train_test_split
x=df.drop('Churn',axis=1)
y=df['Churn']

In [199]:
x_train,x_temp,y_train,y_temp=train_test_split(x,y,test_size=0.3,stratify=y,random_state=42)
x_val,x_test,y_val,y_test=train_test_split(x_temp,y_temp,test_size=0.5,stratify=y_temp,random_state=42)

In [200]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
num_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])
cat_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('one',OneHotEncoder(handle_unknown="ignore",sparse_output=False))
])
preprocessor=ColumnTransformer([
    ('num',num_pipeline,num_cols),
    ('cat',cat_pipeline,cat_cols)
])

In [201]:
scale_pos_weight=(len(y_train)-y_train.sum())/(y_train.sum()+1e-6)
print(scale_pos_weight)

2.7691131477300357


In [202]:
import xgboost as xgb
from sklearn.pipeline import make_pipeline

In [203]:
xgb_clf=xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs= -1,
    scale_pos_weight=scale_pos_weight
)
pipeline=make_pipeline(preprocessor,xgb_clf)

In [204]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 1️⃣ Transform your data using the preprocessor
Xx_train_trans = preprocessor.fit_transform(x_train)
x_val_trans = preprocessor.transform(x_val)

# 2️⃣ Convert to DMatrix (XGBoost's optimized data format)
dtrain = xgb.DMatrix(x_train_trans, label=y_train)
dval = xgb.DMatrix(x_val_trans, label=y_val)

# 3️⃣ Set parameters
params = {
    'objective': 'binary:logistic',   # Binary classification
    'eval_metric': 'auc',             # Metric for performance
    'eta': 0.05,                       # Learning rate
    'max_depth': 6,                    # Tree depth
    'subsample': 0.8,                  # Row sampling
    'colsample_bytree': 0.8,           # Feature sampling
    'scale_pos_weight': scale_pos_weight, # Handle imbalance
    'seed': 42                         # Reproducibility
}

# 4️⃣ List datasets to evaluate during training
watchlist = [(dtrain, 'train'), (dval, 'eval')]

# 5️⃣ Train model with early stopping
model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=300,               # Max number of boosting rounds
    evals=watchlist,                    # Track train & validation
    early_stopping_rounds=20,           # Stop if no improvement for 20 rounds
    verbose_eval=True                   # Show progress
)

# 6️⃣ Predict on validation set
y_pred_proba = model.predict(dval)
y_pred = (y_pred_proba >= 0.5).astype(int)

# 7️⃣ Evaluate
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_val, y_pred))
print(classification_report(y_val, y_pred))


[0]	train-auc:0.85635	eval-auc:0.82490
[1]	train-auc:0.86881	eval-auc:0.83056
[2]	train-auc:0.87610	eval-auc:0.83315
[3]	train-auc:0.88133	eval-auc:0.83711
[4]	train-auc:0.88262	eval-auc:0.83978
[5]	train-auc:0.88509	eval-auc:0.84154
[6]	train-auc:0.88671	eval-auc:0.84176
[7]	train-auc:0.88642	eval-auc:0.84232
[8]	train-auc:0.88821	eval-auc:0.84153
[9]	train-auc:0.88883	eval-auc:0.84154
[10]	train-auc:0.88971	eval-auc:0.84119
[11]	train-auc:0.89055	eval-auc:0.84162
[12]	train-auc:0.89105	eval-auc:0.84318
[13]	train-auc:0.89194	eval-auc:0.84225
[14]	train-auc:0.89253	eval-auc:0.84115
[15]	train-auc:0.89280	eval-auc:0.84186
[16]	train-auc:0.89354	eval-auc:0.84211
[17]	train-auc:0.89424	eval-auc:0.84208
[18]	train-auc:0.89499	eval-auc:0.84273
[19]	train-auc:0.89518	eval-auc:0.84332
[20]	train-auc:0.89543	eval-auc:0.84381
[21]	train-auc:0.89605	eval-auc:0.84390
[22]	train-auc:0.89633	eval-auc:0.84411
[23]	train-auc:0.89690	eval-auc:0.84434
[24]	train-auc:0.89744	eval-auc:0.84436
[25]	train

In [205]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

param_dist = {
    'n_estimators': [100,200,400,800],
    'max_depth': [3,5,7,9],
    'learning_rate': [0.01,0.03,0.05,0.1],
    'subsample': [0.6,0.8,1.0],
    'colsample_bytree': [0.6,0.8,1.0],
    'reg_alpha': [0,0.5,1],
    'reg_lambda': [1,3,5]
}

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
rand = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    n_iter=30,
    scoring='roc_auc',
    cv=kf,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Note: we must transform X_train first
rand.fit(x_train_trans, y_train)
print("Best params:", rand.best_params_)
best_model = rand.best_estimator_


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best params: {'subsample': 0.6, 'reg_lambda': 5, 'reg_alpha': 0.5, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.03, 'colsample_bytree': 0.6}


C:\ana\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:10:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [206]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, average_precision_score,accuracy_score

x_test_trans = preprocessor.transform(x_test)
y_pred_proba = best_model.predict_proba(x_test_trans)[:,1]
y_pred = (y_pred_proba >= 0.5).astype(int)   # adjust threshold later
auc_score=roc_auc_score(y_test, y_pred_proba)

print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

ROC AUC: 0.8447623546245001
              precision    recall  f1-score   support

           0       0.90      0.73      0.81       776
           1       0.52      0.78      0.62       281

    accuracy                           0.75      1057
   macro avg       0.71      0.76      0.72      1057
weighted avg       0.80      0.75      0.76      1057

Confusion Matrix:
 [[570 206]
 [ 61 220]]


In [207]:
from sklearn.metrics import f1_score
best_thresh = 0.5
best_f1 = 0
for t in [i/100 for i in range(30,71)]:
    f1 = f1_score(y_test, (y_pred_proba >= t).astype(int))
    if f1 > best_f1:
        best_f1, best_thresh = f1, t
best_thresh, best_f1

(0.56, 0.6370597243491577)

In [208]:
import shap
# Need feature names after preprocessing
import matplotlib.pyplot as plt
# Helper to get feature names after ColumnTransformer + OneHot
def get_feature_names(preprocessor):
    num_features = num_cols
    cat_features = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_cols)
    return list(num_features) + list(cat_features)

feature_names = get_feature_names(preprocessor)
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(x_test_trans)
import numpy as np

# Calculate mean absolute SHAP value for each feature
mean_abs_shap = np.abs(shap_values).mean(axis=0)

# Create a DataFrame of features and their importance
shap_importance_df = pd.DataFrame({
    'feature': feature_names,
    'mean_abs_shap': mean_abs_shap
})

# Sort by importance
shap_importance_df = shap_importance_df.sort_values(by='mean_abs_shap', ascending=False)

# Select only the top important features
top_features = shap_importance_df.head(10)['feature'].tolist()  # Change 10 to how many you want

# Get column indices for those features
top_feature_indices = [feature_names.index(f) for f in top_features]

# Filter SHAP values and data for only these features
shap_values_top = shap_values[:, top_feature_indices]
x_test_top = x_test_trans[:, top_feature_indices]

# Plot summary for only the important features
shap.summary_plot(
    shap_values_top,
    x_test_top,
    feature_names=top_features,
    color=plt.get_cmap('plasma'),
    show=False
)
plt.savefig("shap_summary_teal_bg.png", dpi=300, bbox_inches='tight',transparent=True)
plt.close()

In [209]:
import joblib
# Save preprocessor + model as a single pipeline for inference
final_pipeline = Pipeline([('preprocessor', preprocessor), ('xgb', best_model)])
joblib.dump(final_pipeline, 'xgb_churn_pipeline.pkl')


['xgb_churn_pipeline.pkl']

In [210]:
pred_df = x_test.copy()
pred_df['actual'] = y_test.values
pred_df['pred_proba'] = y_pred_proba
pred_df['pred'] = (y_pred_proba >= best_thresh).astype(int)
pred_df.reset_index(drop=True, inplace=True)
pred_df.to_csv('churn_predictions_for_powerbi.csv', index=False)

In [211]:
from sklearn.metrics import accuracy_score
acc_score = accuracy_score(y_test, y_pred)
results_df = pd.DataFrame({
    'AUC': [auc_score],
     'ACC':[acc_score]
})

# Save to CSV
results_df.to_csv('model_metrics.csv', index=False)

print("Metrics saved to model_metrics.csv")

Metrics saved to model_metrics.csv


In [213]:
import pandas as pd

# Combine customer details with predictions
customer_table = x_test.copy()

# Add actual churn status
customer_table['Actual_Churn'] = y_test.values

# Add churn probability
customer_table['Churn_Probability'] = y_pred_proba

# Add model's churn prediction (0 = No churn, 1 = Churn)
customer_table['Predicted_Churn'] = (y_pred_proba >= best_thresh).astype(int)
customer_table['High_Risk'] = (customer_table['Churn_Probability'] > 0.7).astype(int)

# Sort customers by highest probability of churn
customer_table = customer_table.sort_values(by='Churn_Probability', ascending=False)
from datetime import datetime

customer_table['Prediction_Date'] = datetime.today().strftime('%Y-%m-%d')
customer_table['CustomerID'] = x_test.index  # or actual ID column
customer_table['Predicted_Churn'] = (y_pred_proba >= best_thresh).astype(int)
customer_table.to_csv("customer_churn_probability_table.csv", index=False)

# Save as CSV for Excel / Power BI
customer_table.to_csv("customer_churn_probability_table.csv", index=False)

print("Table saved as customer_churn_probability_table.csv")


Table saved as customer_churn_probability_table.csv
